In [1]:
import pandas as pd
import numpy as np

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [2]:
train_total = pd.merge(train, train_labels, on='id')

In [3]:
labels = train_labels['label'].drop_duplicates()

In [4]:
train.columns

Index(['id', 'time', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [5]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [7]:
aug_cnt = 1500

for idx, label in labels.iteritems():
    tmps = train_total[train_total['label'] == label].copy()
    while len(train_labels[train_labels['label'] == label]) < aug_cnt:
        id_list = tmps['id'].drop_duplicates()
        print(label,":", len(train_labels[train_labels['label'] == label]))
        for idx, _id in id_list.iteritems():
            if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                tmp = tmps[tmps['id'] == _id].copy()
                tmp['acc_x'] = tmp['acc_x'] + tmp['acc_x'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['acc_y'] = tmp['acc_y'] + tmp['acc_y'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['acc_z'] = tmp['acc_z'] + tmp['acc_z'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['gy_x'] = tmp['gy_x'] + tmp['gy_x'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['gy_y'] = tmp['gy_y'] + tmp['gy_y'].std() * np.random.normal(0.0, 1.0, 600)
                tmp['gy_z'] = tmp['gy_z'] + tmp['gy_z'].std() * np.random.normal(0.0, 1.0, 600)
                newid = train['id'].max() + 1
                tmp['id'] = newid
                train = pd.concat([train, tmp])
                train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)

            if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                for rate in [0.1*i for i in range(5, 15)]:
                    tmp = tmps[tmps['id'] == _id].copy()
                    tmp['acc_x'] = tmp['acc_x']*rate
                    tmp['acc_y'] = tmp['acc_y']*rate
                    tmp['acc_z'] = tmp['acc_z']*rate
                    tmp['gy_x'] = tmp['gy_x']*rate
                    tmp['gy_y'] = tmp['gy_y']*rate
                    tmp['gy_z'] = tmp['gy_z']*rate
                    newid = train['id'].max() + 1
                    tmp['id'] = newid
                    train = pd.concat([train, tmp])
                    train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)

            if len(train_labels[train_labels['label'] == label]) < aug_cnt:
                tmp = tmps[tmps['id'] == _id].copy()
                tmp.sort_values(by='time', ascending=False)
                newid = train['id'].max() + 1
                tmp['id'] = newid
                train = pd.concat([train, tmp])
                train_labels = train_labels.append({'id': newid, 'label': label}, ignore_index=True)
   
    print(label)


37 : 35
37 : 455
37 : 875
37 : 1295


KeyboardInterrupt: 